In [ ]:
import os
import numpy as np
import scipy.io as sio
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.utils.data
from torch.autograd import Variable
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from tqdm import tqdm

# Configuration
CONFIG = {
    'workers': 0,
    'batch_size': 200,
    'model_path': 'model/best_model.pth',  # path to trained model
    'data_path': 'data',                   # path to dataset directory
    'cuda': torch.cuda.is_available(),
    'pca_components': 3
}

if CONFIG['cuda']:
    torch.cuda.set_device(0)
    cudnn.benchmark = False

class Discriminator(nn.Module):
    def __init__(self, ndf, nc, nb_label):
        super(Discriminator, self).__init__()
        self.LeakyReLU = nn.LeakyReLU(0.2, inplace=True)
        self.conv1 = nn.Conv2d(nc, ndf, 3, 1, 0, bias=False)
        self.BatchNorm1 = nn.BatchNorm2d(ndf)
        self.conv2 = nn.Conv2d(ndf, ndf * 2, 3, 1, 0, bias=False)
        self.BatchNorm2 = nn.BatchNorm2d(ndf * 2)
        self.conv3 = nn.Conv2d(ndf * 2, ndf * 4, 3, 1, 0, bias=False)
        self.BatchNorm3 = nn.BatchNorm2d(ndf * 4)
        self.conv4 = nn.Conv2d(ndf * 4, ndf * 8, 3, 1, 0, bias=False)
        self.BatchNorm4 = nn.BatchNorm2d(ndf * 8)
        self.conv5 = nn.Conv2d(ndf * 8, ndf * 2, 3, 1, 0, bias=False)
        self.aux_linear = nn.Linear(ndf * 2, nb_label + 1)
        self.softmax = nn.LogSoftmax(dim=-1)
        self.ndf = ndf

    def forward(self, input):
        x = self.conv1(input)
        x = self.LeakyReLU(x)
        x = self.conv2(x)
        x = self.BatchNorm2(x)
        x = self.LeakyReLU(x)
        x = self.conv3(x)
        x = self.BatchNorm3(x)
        x = self.LeakyReLU(x)
        x = self.conv4(x)
        x = self.BatchNorm4(x)
        x = self.LeakyReLU(x)
        x = self.conv5(x)
        x = x.view(-1, self.ndf * 2)
        c = self.aux_linear(x)
        c = self.softmax(c)
        return c

def applyPCA(X, numComponents):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0], X.shape[1], numComponents))
    return newX

def padWithZeros(X, margin=5):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2 * margin, X.shape[2]))
    newX[margin:X.shape[0] + margin, margin:X.shape[1] + margin, :] = X
    return newX

def createImageCubes(X, y, windowSize=11, removeZeroLabels=True):
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    patchesData = []
    patchesLabels = []
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]
            label = y[r-margin, c-margin]
            if label > 0 or not removeZeroLabels:
                patchesData.append(patch)
                patchesLabels.append(label)
    return np.array(patchesData), np.array(patchesLabels)

class TestDS(torch.utils.data.Dataset):
    def __init__(self, x_data, y_data):
        self.len = x_data.shape[0]
        self.x_data = torch.FloatTensor(x_data)
        self.y_data = torch.LongTensor(y_data)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

def load_data(data_path):
    matfn1 = os.path.join(data_path, 'Indian_pines_corrected.mat')
    data1 = sio.loadmat(matfn1)
    X = data1['indian_pines_corrected']
    matfn2 = os.path.join(data_path, 'Indian_pines_gt.mat')
    data2 = sio.loadmat(matfn2)
    y = data2['indian_pines_gt']
    return X, y

def prepare_data(X, y, pca_components=3):
    X_pca = applyPCA(X, numComponents=pca_components)
    patches, labels = createImageCubes(X_pca, y, windowSize=11)
    labels = labels - 1
    return patches.transpose(0, 3, 1, 2).astype('float32'), labels

def kappa(confusion_matrix):
    n = np.sum(confusion_matrix)
    sum_po = np.sum(np.diag(confusion_matrix))
    po = sum_po / n
    pe = np.sum(np.sum(confusion_matrix, axis=0) * np.sum(confusion_matrix, axis=1)) / (n * n)
    return (po - pe) / (1 - pe)

def evaluate_model(model, test_loader, criterion, num_classes, device):
    model.eval()
    test_loss = 0
    correct = 0
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for data, target in tqdm(test_loader, desc='Evaluating'):
            if device == 'cuda':
                data, target = data.cuda(), target.cuda()
            
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.max(1)[1]
            all_preds.extend(pred.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
            correct += pred.eq(target).sum().item()

    test_loss /= len(test_loader)
    accuracy = 100. * correct / len(test_loader.dataset)
    
    # Calculate confusion matrix and metrics
    conf_matrix = confusion_matrix(np.array(all_targets), np.array(all_preds))
    class_acc = np.diag(conf_matrix) / np.sum(conf_matrix, axis=1)
    average_accuracy = np.mean(class_acc)
    kappa_score = kappa(conf_matrix)
    
    return {
        'test_loss': test_loss,
        'accuracy': accuracy,
        'average_accuracy': average_accuracy * 100,
        'kappa': kappa_score,
        'confusion_matrix': conf_matrix,
        'class_accuracies': class_acc * 100
    }

def main():
    # Load and prepare data
    X, y = load_data(CONFIG['data_path'])
    X_test, y_test = prepare_data(X, y, CONFIG['pca_components'])
    
    # Create test dataset and loader
    testset = TestDS(X_test, y_test)
    test_loader = torch.utils.data.DataLoader(
        testset, 
        batch_size=CONFIG['batch_size'],
        shuffle=False,
        num_workers=CONFIG['workers']
    )
    
    # Load model
    checkpoint = torch.load(CONFIG['model_path'])
    num_classes = len(np.unique(y_test))
    model = Discriminator(ndf=64, nc=CONFIG['pca_components'], nb_label=num_classes)
    model.load_state_dict(checkpoint['state_dict_D'])
    
    device = 'cuda' if CONFIG['cuda'] else 'cpu'
    if CONFIG['cuda']:
        model = model.cuda()
    
    criterion = nn.NLLLoss()
    if CONFIG['cuda']:
        criterion = criterion.cuda()
    
    # Evaluate model
    results = evaluate_model(model, test_loader, criterion, num_classes, device)
    
    # Print results
    print("\nEvaluation Results:")
    print(f"Test Loss: {results['test_loss']:.4f}")
    print(f"Overall Accuracy: {results['accuracy']:.2f}%")
    print(f"Average Accuracy: {results['average_accuracy']:.2f}%")
    print(f"Kappa Coefficient: {results['kappa']:.4f}")
    
    print("\nClass-wise Accuracies:")
    for i, acc in enumerate(results['class_accuracies']):
        print(f"Class {i+1}: {acc:.2f}%")
    
    # Save confusion matrix
    np.save('confusion_matrix.npy', results['confusion_matrix'])
    print("\nConfusion matrix saved as 'confusion_matrix.npy'")

if __name__ == '__main__':
    main()